In [ ]:
# 提取PFR并降分辨率到0.1度
import xarray as xr
import numpy as np
import os

# 1. 读取原始的.nc文件
var = 'PFR'
dir_l = os.listdir("D:/Permafrost/mask")
for nc_d in dir_l:
    ds = xr.open_dataset(f"D:/Permafrost/mask/{nc_d}")
    data = ds[var]
    coarsen_data = data.coarsen(lat=10, lon=10, boundary='trim').mean()
    # 4. 创建新的DataArray，并将其加入到新的Dataset中
    new_ds = xr.Dataset(
        {var: coarsen_data},
        coords={
            'lat': coarsen_data.coords['lat'],
            'lon': coarsen_data.coords['lon']
        }
    )

    # 5. 将新的Dataset保存为新的.nc文件
    new_file_path = f"D:/Permafrost/mask_re/{nc_d[-14:-10]}.nc"  # 你可以自定义新的输出文件路径
    new_ds.to_netcdf(new_file_path)
    print(f"Saved coarsened data to {new_file_path}")
    # 6. 关闭打开的Dataset
    ds.close()
    new_ds.close()

In [ ]:
# 2. 对每5个文件求均值并保存为新的.nc文件
import xarray as xr
import numpy as np
import os

# 假设你的 25 个 nc 文件都保存在某个目录下
input_dir = 'D:/Permafrost/mask_re/re'  # 替换为你的 nc 文件所在的目录
output_dir = 'D:/Permafrost/mask_re'  # 替换为你想保存输出文件的目录

# 生成文件路径列表，确保文件是按正确的顺序排列的
file_paths = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.nc')])

# 确保你有 25 个文件
assert len(file_paths) == 25, "文件数量应为 25 个！"

# 对每 5 个文件求一次均值
for i in range(0, len(file_paths), 5):
    # 读取 5 个文件
    ds_list = [xr.open_dataset(file_paths[j]) for j in range(i, i+5)]
    print(ds_list)
    # # 提取每个文件中的PFR变量，假设变量名为 'PFR'
    # pfr_data_list = [ds['PFR'] for ds in ds_list]

    # # 计算每 5 个文件的均值
    # pfr_mean = np.mean(pfr_data_list, axis=0)

    # # 创建一个新的 Dataset 来保存均值数据
    # new_ds = xr.Dataset(
    #     {'PFR_mean': (['lat', 'lon'], pfr_mean[0])},  # 假设 'lat' 和 'lon' 是维度
    #     coords={'lat': ds_list[0].lat, 'lon': ds_list[0].lon}
    # )
    
    # # 保存为新的.nc文件
    # output_filename = f'pfr_mean_{i//5 + 1}.nc'
    # new_ds.to_netcdf(os.path.join(output_dir, output_filename))

    # # 关闭当前处理的 Dataset
    # for ds in ds_list:
    #     ds.close()

    # # 关闭新创建的 Dataset
    # new_ds.close()

print("处理完毕！")


In [ ]:
# 3. 读取均值文件并绘图
import xarray as xr
import matplotlib.pyplot as plt
import os

# 输出目录：包含你计算均值后保存的.nc文件
output_dir = 'D:/Permafrost/mask_re/mean'  # 替换为实际输出目录
output_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.nc')])

# 对每个保存均值的.nc文件绘制图像
for file in output_files:
    # 读取.nc文件
    file_path = os.path.join(output_dir, file)
    ds = xr.open_dataset(file_path)
    
    # 获取 PFR_mean 数据
    pfr_mean = ds['PFR_mean']
    
    # 绘制数据
    plt.figure(figsize=(12, 6))  # 设置图像大小
    pfr_mean.plot(cmap='Blues_r')  # 使用viridis色图进行绘制
    plt.title(f'PFR_mean from {file}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    # 保存图像
    plt.show()
    # plt.savefig(f"{file.replace('.nc', '.png')}", dpi=300)
    plt.close()
    # 关闭 Dataset
    ds.close()
    break

In [ ]:
# 计算大于90和小于等于90的格点数比例
import xarray as xr
import os
import numpy as np

# 均值文件所在目录
data_dir = "D:/Permafrost/mask_re/resm"   # 替换成你自己的目录
files = sorted([f for f in os.listdir(data_dir) if f.endswith(".nc")])

results = []
T_p = []
N_P = []
for f in files:
    path = os.path.join(data_dir, f)
    ds = xr.open_dataset(path)

    da = ds["PFR"]

    # 统计大于90的格点数和小于等于90的格点数
    count_gt90 = (da >= 90).sum().item()
    count_le90 = (da < 90).sum().item()

    results.append({
        "file": f,
        "gt90": count_gt90,
        "le90": count_le90
    })
    T_p.append(count_gt90)
    N_P.append(count_le90)
    ds.close()
T_p = np.array(T_p)
N_P = np.array(N_P)
# 打印结果
# for r in results:
#     print(f"{r['file']}: >90 = {round(r['gt90']/(r['gt90'] + r['le90']), 3)}, <=90 = {round(r['le90']/(r['gt90'] + r['le90']), 3)}")

In [ ]:
# 提取多个变量并保存为新的.nc文件  v10  str strd  ssd ssrd #"swvl1", "swvl2", "swvl3", "swvl4"   "stl1", "stl2", "stl3", "stl4"   "e" 蒸散 "t2m" 2米气温 "tp" 总降雨 "sp" 气压
import xarray as xr
import numpy as np
import os
# 原始文件路径
input_file = "D:/Permafrost/temp/data_stream-moda.nc"
# 需要提取的变量名列表（根据你的数据修改）
vars_to_extract = 'sde' 
# 输出文件路径
output_file = f"D:/Permafrost/temp/var/{vars_to_extract}.nc"
# 打开原始 nc 文件
ds = xr.open_dataset(input_file)
# 选择需要的变量
ds_new = ds[vars_to_extract]
ds_new1 = ds_new.isel(latitude=slice(None, 600))
# 设定新的经纬度范围，这里假设你想要将经度从 -180 到 180，纬度从 -90 到 90
# 你可以根据实际需求调整标准值的范围和步长
lon_standard = np.linspace(-180, 180, 3600)  # 根据原始经度维度大小设定
lat_standard = np.linspace(25, 85, 600)    # 根据原始纬度维度大小设定
lon_standard = np.round(lon_standard,1)
lat_standard = np.round(lat_standard,1)
# 替换经纬度的坐标值  latitude和longitude
ds_new2 = ds_new1.assign_coords(longitude=lon_standard, latitude=lat_standard[::-1])
# 保存为新的 nc 文件
ds_new2.to_netcdf(output_file)
# 关闭数据集
ds.close()
ds_new.close()
ds_new1.close()
ds_new2.close()
print("提取并保存完成！")

In [ ]:
# 修剪.nc文件的经纬度维度 600
import xarray as xr

# 打开你的原始 nc 文件
input_file = "D:/Permafrost/temp/data_stream-moda.nc"
ds = xr.open_dataset(input_file)

# 检查原始维度大小
print(ds.dims)  # 检查维度

# 假设是删除经度方向的最后一个点
# 如果是删除纬度，可以类似操作，替换 lat 为 lon
ds_trimmed = ds.isel(latitude=slice(None, 600))  # 删除纬度的最后一个点

# 检查修改后的维度大小
print(ds_trimmed.dims)

# 保存为新的 nc 文件
output_file = "D:/Permafrost/temp/data_stream-moda_modified.nc"
ds_trimmed.to_netcdf(output_file)

# 关闭数据集
ds.close()
ds_trimmed.close()

print(f"Saved trimmed file as {output_file}")


In [ ]:
# 修改经纬度维度为标准值
import xarray as xr
import numpy as np

# 读取原始的 NetCDF 文件
input_file = "D:/Permafrost/temp/1997_modified.nc"
ds = xr.open_dataset(input_file)

# 设定新的经纬度范围，这里假设你想要将经度从 -180 到 180，纬度从 -90 到 90
# 你可以根据实际需求调整标准值的范围和步长
lon_standard = np.linspace(-180, 180, 3600)  # 根据原始经度维度大小设定
lat_standard = np.linspace(25, 85, 600)    # 根据原始纬度维度大小设定
lon_standard = np.round(lon_standard,1)
lat_standard = np.round(lat_standard,1)
# 替换经纬度的坐标值  latitude和longitude
ds = ds.assign_coords(longitude=lon_standard, latitude=lat_standard[::-1])

# 保存为新的 NetCDF 文件
output_file = "D:/Permafrost/temp/1997_modified111.nc"
ds.to_netcdf(output_file)

# 关闭数据集
ds.close()

print(f"New file saved as {output_file}")


In [ ]:
# # swvl 的批量处理掩膜
# import xarray as xr
# import numpy as np
# import os

# # ===== 配置 =====
# monthly_file = "D:/Permafrost/temp/swvl_1_4.nc"   # 25年×12月的原始月度数据
# output_dir   = "D:/Permafrost/temp/swvl"             # 按年掩膜后输出的目录
# mask_dir     = "D:/Permafrost/mask_re/resm"         # 存放每年掩膜的目录
# mask_pattern = "{year}.nc"            # 每年的掩膜文件名模式
# mask_var     = "PFR"                      # 掩膜变量名（2D: lat×lon）
# vars_to_keep = ['swvl1', 'swvl2', 'swvl3', 'swvl4']   # 想只保留部分变量可填列表，如 ["PFR","T","RH"]；None=全部数据变量

# # 打开月度总文件
# ds = xr.open_dataset(monthly_file)
# ds = ds[vars_to_keep]

# # 按年份分组
# for year, ds_year in ds.groupby("valid_time.year"):
#     year = int(year)
#     # 读取该年的掩膜
#     mask_path = os.path.join(mask_dir, mask_pattern.format(year=year))
#     mds = xr.open_dataset(mask_path)

#     # 取出掩膜 DataArray
#     mask = mds[mask_var]

#     # 如果掩膜不是2D（例如带有time维），尝试取第一时次或做聚合
#     if "time" in mask.dims:
#         mask = mask.isel(time=0)

#     # 广播掩膜到(time, lat, lon)，并应用掩膜
#     ds_masked = ds_year.where(mask >= 0)

#     # 可选：保留/拷贝一些全局属性
#     ds_masked.attrs = ds_year.attrs

#     # 保存
#     out_path = os.path.join(output_dir, f"swvl_1_4_{year}_masked.nc")
#     ds_masked.to_netcdf(out_path)
#     print(f"Saved: {out_path}")

#     # 关闭掩膜文件
#     mds.close()
# # 关闭原始数据
# ds.close()


In [ ]:
# 每年的掩膜处理
import xarray as xr
import numpy as np
import os
# for f in ["swvl1", "swvl2", "swvl3", "swvl4"]:
# ===== 配置 =====  #"e" 蒸散 "t2m" 2米气温 "tp" 总降雨 "sp" 气压
f = 'sde'
monthly_file = f"D:/Permafrost/temp/var/{f}.nc"   # 25年×12月的原始月度数据
output_dir   = f"D:/Permafrost/temp/var/{f}"          # 按年掩膜后输出的目录
mask_dir     = "D:/Permafrost/mask_re/resm"         # 存放每年掩膜的目录
mask_pattern = "{year}.nc"       # 每年的掩膜文件名模式
mask_var     = "PFR"         # 掩膜变量名（2D: lat×lon）
vars_to_keep = f'{f}'    # 想只保留部分变量可填列表，如 ["PFR","T","RH"]；None=全部数据变量['swvl4', 'swvl4', 'swvl4', 'swvl4']

# 打开月度总文件
ds = xr.open_dataset(monthly_file)
ds = ds[vars_to_keep]

# 按年份分组
for year, ds_year in ds.groupby("valid_time.year"):
    year = int(year)
    # 读取该年的掩膜
    mask_path = os.path.join(mask_dir, mask_pattern.format(year=year))
    mds = xr.open_dataset(mask_path)

    # 取出掩膜 DataArray
    mask = mds[mask_var]

    # 如果掩膜不是2D（例如带有time维），尝试取第一时次或做聚合
    if "time" in mask.dims:
        mask = mask.isel(time=0)

    # 广播掩膜到(time, lat, lon)，并应用掩膜
    ds_masked = ds_year.where(mask >= 0)

    # 可选：保留/拷贝一些全局属性
    ds_masked.attrs = ds_year.attrs

    # 保存
    out_path = os.path.join(output_dir, f"{f}_{year}_masked.nc")
    ds_masked.to_netcdf(out_path)
    # 关闭掩膜文件
    mds.close()
# 关闭原始数据
print(f"Finished processing {f}")
ds.close()


In [ ]:
# 批量修改经纬度维度为标准值
import xarray as xr
import numpy as np
import os 
dir_l = os.listdir('D:/Permafrost/mask_re/res')
for nc_d in dir_l:
    # 读取原始的 NetCDF 文件
    input_file = f"D:/Permafrost/mask_re/res/{nc_d}"
    ds = xr.open_dataset(input_file)
    # 你可以根据实际需求调整标准值的范围和步长
    lon_standard = np.linspace(-180, 180, 3600)  # 根据原始经度维度大小设定
    lat_standard = np.linspace(25, 85, 600)    # 根据原始纬度维度大小设定
    lon_standard = np.round(lon_standard,1)
    lat_standard = np.round(lat_standard,1)
    # 替换经纬度的坐标值  latitude和longitude
    ds = ds.assign_coords(longitude=lon_standard, latitude=lat_standard)

    # 保存为新的 NetCDF 文件
    output_file = f"D:/Permafrost/mask_re/resm/{nc_d}.nc"
    ds.to_netcdf(output_file)

    # 关闭数据集
    ds.close()

    print(f"New file saved as {output_file}")


In [ ]:
# 读取某变量月均数据并保存为Excel
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
import xarray as xr
import matplotlib.pyplot as plt

# for f in ["swvl1", "swvl2", "swvl3", "swvl4"]: # ["stl1", "stl2", "stl3", "stl4"]  "e" 蒸散 "t2m" 2米气温 "tp" 总降雨 "sp" 气压
f = 'sde'
# ===== 配置 =====
input_dir = f"D:/Permafrost/temp/var/{f}"  # 存储每年月均数据的文件夹
vars_to_keep = f  # 只保留需要的变量
# 获取文件夹中的所有文件，按年份排序
files = sorted([f for f in os.listdir(input_dir) if f.endswith('.nc')])
# 初始化一个空列表，用于保存所有读取的数据
all_monthly_data = []
data_all = []
# 逐个读取每个文件
for file in files:
    # 构造文件路径
    file_path = os.path.join(input_dir, file)
    # 打开文件
    ds = xr.open_dataset(file_path)
    ds = ds[vars_to_keep]  # 保留所需的变量
    # 计算该年的月均值，并添加到列表中
    monthly_mean = ds.mean(dim=['latitude', 'longitude'])  # 假设time是月份维度   'latitude', 'longitude'
    all_monthly_data.append(np.array(monthly_mean))
    # 关闭数据集
    ds.close()
combined_data = np.concatenate(all_monthly_data)
data_all.append(combined_data)
import pandas as pd
# 生成时间序列索引
dates = pd.date_range(start="1997-01", periods=300, freq="M")
# 组合成 DataFrame
df = pd.DataFrame({
    "Date": dates,
    f"{f}": combined_data
})
# 保存到 Excel
df.to_excel(f"D:/Permafrost/temp/var/{f}/{f}.xlsx", index=False)
print(f"Saved data for {f} to Excel.")

In [ ]:
# 计算相对湿度、水蒸气压
import pandas as pd

# 读取 Excel 文件
t2m = pd.read_excel('D:/Permafrost/EN_MASK/var/ecl/t2m.xlsx', index_col=0)-273.15  # 读取温度数据并转换为摄氏度
# 露点温度数据假设已经存储在另一个 Excel 文件中
t2m_dpt = pd.read_excel('D:/Permafrost/EN_MASK/var/ecl/d2m.xlsx', index_col=0)-273.15  # 读取露点温度数据并转换为摄氏度

# 计算相对湿度（RH）
def calculate_saturation_vapor_pressure(temperature_C):
    """
    计算给定温度下的饱和水蒸气压（单位：hPa）
    使用 Tetens 公式
    :param temperature_C: 温度，单位：摄氏度
    :return: 水蒸气饱和压力，单位：hPa
    """
    return 6.11 * 10**(7.5 * temperature_C / (temperature_C + 237.3))

def calculate_relative_humidity(temperature_C, dew_point_C):
    """
    计算相对湿度
    :param temperature_C: 当前空气温度，单位：摄氏度
    :param dew_point_C: 露点温度，单位：摄氏度
    :return: 相对湿度，单位：百分比
    """
    # 计算当前温度下的水蒸气压力 E(T)
    E_T = calculate_saturation_vapor_pressure(temperature_C)
    
    # 计算露点温度下的饱和水蒸气压力 E(T_d)
    E_Td = calculate_saturation_vapor_pressure(dew_point_C)
    
    # 计算相对湿度 RH
    RH = (E_Td / E_T) * 100
    return RH

# 假设 t2m 和 t2m_dpt 的索引是日期
relative_humidity = []

# 逐行计算相对湿度
for date in t2m.index:
    temperature = t2m.loc[date].values[0]  # 获取当天的温度数据
    dew_point = t2m_dpt.loc[date].values[0]  # 获取当天的露点温度数据
    relative_humidity.append(calculate_relative_humidity(temperature, dew_point))

# 将结果转为excel
pd.DataFrame(relative_humidity, index=t2m.index, columns=['Relative Humidity']).to_excel('D:/Permafrost/EN_MASK/var/ecl/relative_humidity.xlsx')


In [ ]:
#绘制相对湿度图
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
# 读取相对湿度数据
rh = pd.read_excel('D:/Permafrost/temp/relative_humidity.xlsx', index_col=0)
rh.index = pd.to_datetime(rh.index)
# 绘制相对湿度图
plt.figure(figsize=(12, 6))
plt.plot(rh.index, rh['Relative Humidity'], label='Relative Humidity', color='blue')
plt.xlabel('Date')
plt.ylabel('Relative Humidity (%)')
plt.title('Relative Humidity Over Time')
plt.grid(True)
plt.legend()
plt.show()

